In [ ]:
#import important modules
import pymongo as pm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from pymongo import MongoClient
import json
import pickle
import re
import pprint
import datetime
import statistics as st

In [ ]:
with open('editeddata/conversations.json', 'r') as convo_json:
    convos = json.load(convo_json)

In [ ]:
client = MongoClient('localhost', 27017)
mydb = client.tweets
col = mydb['_tweets']

In [ ]:
airlines = {'KLM':56377143, 
            'AirFrance': 106062176, 
            'British_Airways': 18332190, 
            'AmericanAir': 22536055, 
           'Lufthansa': 124476322, 
           'AirBerlin': 26223583, 
           'AirBerlinAssist': 2182373406, 
           'easyJet': 38676903, 
           'RyanAir': 1542862735, 
           'SingaporeAir': 253340062, 
           'Qantas': 218730857, 
           'EtihadAirways': 45621423, 
           'VirginAtlantic': 20626359}

In [ ]:
air_count = pickle.load(open("save.p", "rb"))

Get the ratio of replied tweets for every airline

In [ ]:
for name in airlines:
    if len(air_count[name]['ids']) != 0:
        conv_tweets = len(air_count[name]['ids']) - len(list(col.find({'user.id': airlines[name], 'in_reply_to_user_id':'none'})))
        a = len(list(col.find({'user.id' : airlines[name], 'in_reply_to_user_id' :{'$ne': 'none'}})))
        air_count[name]['rep_to_all'] = a / conv_tweets
        if a != 0:
            air_count[name]['rec_to_rep'] = (conv_tweets - a) / a 
        else:
            air_count[name]['rec_to_rep'] = conv_tweets
    else:
        air_count[name]['rep_to_all'] = 0
        air_count[name]['rec_to_rep'] = 0

Received over reply tweets. 
We exclude AirBerlin (which didn't reply at all to its cutomers) and AirBerlinAssist (which doesn't have any tweets associated with them)

In [ ]:
reps = pd.DataFrame(columns=['airline', 'rec_to_rep_ratio'])
reps['airline'] = [i for i in airlines.keys() if i not in 'AirBerlinAssist']
reps['rec_to_rep_ratio'] = [air_count[name]['rec_to_rep'] for name in airlines if name not in 'AirBerlinAssist']
reps_sort = reps.sort_values(by='rec_to_rep_ratio', ascending=False)
reps_sort = reps_sort.reset_index(drop=True)
reps_sort

The point is to be low on the table as that indicates a low number of received tweets per a reply

In [ ]:
plt.figure(figsize=(12,5))
plt.bar(reps_sort['airline'], reps_sort['rec_to_rep_ratio'], color='green')
plt.xticks(rotation='45')
plt.title('Ratio of received to reply tweets per airline', size=15)
plt.ylabel('Number of received per a reply', size=12);

In [ ]:
print('SingaporeAir:', air_count['SingaporeAir']['rec_to_rep'], '\n', 'KLM:', air_count['KLM']['rec_to_rep'], '\n' 'Mean:', st.mean(list(reps['rec_to_rep_ratio'])))


In [ ]:
rep_ratios = pd.DataFrame(columns=['airline', 'rep_to_all'])
rep_ratios['airline'] = [i for i in airlines.keys()]
rep_ratios['rep_to_all'] = [air_count[name]['rep_to_all'] for name in airlines.keys()]
rep_ratios_sort = rep_ratios.sort_values(by='rep_to_all', ascending=False)
rep_ratios_sort

In [ ]:
plt.figure(figsize=(12,5))
plt.bar(rep_ratios_sort['airline'], rep_ratios_sort['rep_to_all'])
plt.xticks(rotation='45')
plt.title('Ratio of reply tweets per airline', size=15);

Replies over all tweets

In [ ]:
print('KLM:', air_count['KLM']['rep_to_all'], '\n', 'Mean:',  st.mean(list(rep_ratios['rep_to_all'])))


Number of tweets received from users normalized on the number of followers -> THIS WON'T BE INCLUDED

In [ ]:
for name in airlines:
    if name != 'AirBerlin' and name != 'AirBerlinAssist':
        air_count[name]['followers'] = col.find_one({'user.id': airlines[name]})['user']['followers_count']
        air_count[name]['norm_tweets'] = (air_count[name]['nr']*(1-air_count[name]['rep_ratio']))/air_count[name]['followers']
    else:
        air_count[name]['followers'] = 'unknown'
        air_count[name]['norm_tweets'] = 'unknown'

Note: AirBerlin and AirBerlinAssist are not included since we don't have any tweets from them, hence no information on the number of followers.

In [ ]:
for name in airlines:
    print(air_count[name]['norm_tweets'])

We get the ratio of tweets customers post to the number of followers a company has.
This is problematic since we are not sure the people who tweeted about an airline are actually their followers.

In [ ]:
import calendar

In [ ]:
rec = []
d = []
w = []
k = 0
for i in air_count['KLM']['ids']:
    k += 1
    t = col.find_one({'id': i})
    if t['user']['id'] not in airlines.values():
        rec.append(i)
        datetime_obj = datetime.date.fromtimestamp(int(t['timestamp_ms'])/1000)
        d.append(datetime_obj.strftime('%Y-%m-%d'))
        w.append(datetime_obj.weekday())

In [ ]:
klm = pd.DataFrame({'rec_ids' : rec, 'date': d, 'weekday': w})